In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

CREATING THE WIKIPEDIA TOOL TO USE THE AGENT

In [8]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,top_k_chars=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [9]:
wiki.name

'wikipedia'

CREATING TOOL FOR WEBBASED SEARCH

In [ ]:
# tamilnadu_retriever.py
import os
os.environ["USER_AGENT"] = "MyLangChainBot/1.0 (contact@example.com)"

  

# Step 1: Load webpage
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Tamil_Nadu")  
print(" Docs loaded")

# Step 2: Split into chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(docs[:1])  
print(" Docs split into chunks")

✅ Docs loaded
✅ Docs split into chunks


In [ ]:


# Step 3: Create Gemini embeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embedding = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",     
    task_type="retrieval_document"    
)

# Step 4: Store embeddings in FAISS
from langchain.vectorstores import FAISS
db = FAISS.from_documents(split_docs, embedding)
retriever = db.as_retriever()
print(" Retriever ready")




✅ Retriever ready


In [28]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever,"tamilnadusearch","Search for any information about tamilnadu use this tool ")

In [29]:
retriever_tool.name

'tamilnadusearch'

CREATING  TOOL FOR RESEARCH PAPERS 

In [30]:
#3. Research Paper
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=10)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [31]:
arxiv.name

'arxiv'

In [32]:
tools = [wiki,retriever_tool,arxiv]

In [33]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Vigneshwaran G\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)),
 Tool(name='tamilnadusearch', description='Search for any information about tamilnadu use this tool ', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000020FFF98F4C0>, retriever=VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020FFFF679B0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000020FFF

CREATING AGENT TO PERFORM THE TOOL CALLING 

In [41]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()

llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash",temperature=0)

In [ ]:

prompt = hub.pull("hwchase17/openai-functions-agent")

c:\Users\Vigneshwaran G\AppData\Local\Programs\Python\Python312\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [43]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
agent = create_tool_calling_agent(llm,tools,prompt)

In [44]:
agent_executor = AgentExecutor(agent =agent,tools = tools , verbose=True)


In [46]:
agent_executor.invoke({"input": "give me some facts on cricket"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'cricket'}`


Page: Cricket
Summary: Cricket is a bat-and-ball game that is played between two teams of eleven players on a field, at the centre of which is a 22-yard (20-metre; 66-foot) pitch with a wicket at each end, each comprising two bails (small sticks) balanced on three stumps. Two players from the batting team, the striker and nonstriker, stand in front of either wicket holding bats, while one player from the fielding team, the bowler, bowls the ball toward the striker's wicket from the opposite end of the pitch. The striker's goal is to hit the bowled ball with the bat and then switch places with the nonstriker, with the batting team scoring one run for each of these swaps. Runs are also scored when the ball reaches the boundary of the field or when the ball is bowled illegally.
The fielding team aims to prevent runs by dismissing batters (so they are "out"). Dismissal can occur in various ways, in

{'input': 'give me some facts on cricket',
 'output': 'Cricket is a bat-and-ball game played between two teams of eleven players.  The game takes place on a field with a 22-yard pitch and a wicket at each end.  The goal is to score runs by hitting the ball bowled by the opposing team and running between the wickets, or by hitting boundaries.  The fielding team tries to prevent runs and dismiss batsmen.  Different formats of cricket exist, ranging from five-day Test matches to shorter Twenty20 matches.  The sport originated in South East England in the 16th century and spread globally with the British Empire.  The International Cricket Council (ICC) governs the sport.\n'}